![car](car.jpg)

Insurance companies invest a lot of [time and money](https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf) into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries insurance it is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

Knowing all of this, On the Road car insurance have requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to identify the single feature that results in the best performing model, as measured by accuracy, so they can start with a simple model in production.

They have supplied you with their customer data as a csv file called `car_insurance.csv`, along with a table detailing the column names and descriptions below.



## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-15</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code | 
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client | 
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

In [1]:
# Import required modules
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit

# Start coding!

#Read file and store in DataFrame
file_name = "car_insurance.csv"
insurance = pd.read_csv(file_name)
print(insurance.head())
print(insurance.info())

# Define logistic regression function
def logit_reg_acc(model, column):
    mdl = logit(model, data=insurance).fit()
    conf_matrix = mdl.pred_table()
    acc = (conf_matrix[0,0] + conf_matrix[1,1]) / (conf_matrix[0,0] + conf_matrix[1,1] + conf_matrix[1,0] + conf_matrix[0,1])
    return acc

       id  age  gender  ... duis past_accidents outcome
0  569520    3       0  ...    0              0     0.0
1  750365    0       1  ...    0              0     1.0
2  199901    0       0  ...    0              0     0.0
3  478866    0       1  ...    0              0     0.0
4  731664    1       1  ...    0              1     1.0

[5 rows x 18 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10000 non-null  int64  
 1   age                  10000 non-null  int64  
 2   gender               10000 non-null  int64  
 3   driving_experience   10000 non-null  object 
 4   education            10000 non-null  object 
 5   income               10000 non-null  object 
 6   credit_score         9018 non-null   float64
 7   vehicle_ownership    10000 non-null  float64
 8   vehicle_year         10000 n

In [2]:
# Set transformation dictionaries
dict_driving_experience = {
    '0-9y': 0,
    '10-19y': 1,
    '20-29y': 2,
    '30y+': 3
}

dict_education = {
    'none': 0,
    'high school': 1,
    'university': 2
}

dict_income = {
    'poverty': 0,
    'working class': 1,
    'middle class': 2,
    'upper class': 3
}

dict_vehicle_year = {
    'before 2015': 0,
    'after 2015': 1
}

dict_vehicle_type = {
    'sedan': 0,
    'sports car': 1
}

# Set list of columns to replace values
cols_replace = [
    'driving_experience',
    'education',
    'income',
    'vehicle_year',
    'vehicle_type'
]

# Set list of dictionaries
list_dicts = [
    dict_driving_experience,
    dict_education,
    dict_income,
    dict_vehicle_year,
    dict_vehicle_type
]

# Replace values
for col in cols_replace:
    dict_replace = list_dicts[cols_replace.index(col)]
    insurance[col] = insurance[col].replace(dict_replace.keys(), dict_replace.values())
    insurance[col] = insurance[col].astype(int)
    
# Convert other columns to integer type
float_columns = [
    'vehicle_ownership',
    'married',
    'children',
    'outcome'
]

for col in float_columns:
    insurance[col] = insurance[col].astype(int)

In [3]:
# Replace missing values
insurance['annual_mileage'].fillna(insurance['annual_mileage'].mean(), inplace=True)
insurance['credit_score'].fillna(insurance['credit_score'].mean(), inplace=True)

In [4]:
# List models to predict
model_variables = insurance.columns[1:17]
model_dict = {}

for item in model_variables:
    model_dict[item] = f'outcome ~ {item}'

# Record accuracy of models
acc_list = [logit_reg_acc(v, k) for k, v in model_dict.items()]
best_feature_df = pd.DataFrame({'best_feature': np.array(model_dict.keys()),
                                'best_accuracy': np.array(acc_list)})
best_feature_df = best_feature_df[best_feature_df['best_accuracy'] == best_feature_df['best_accuracy'].max()]
print(best_feature_df)

Optimization terminated successfully.
         Current function value: 0.511794
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.615951
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.467390
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.603848
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.531580
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.572557
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.552412
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.572668
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.586659
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.595431
  